In [1]:
import pandas as pd
import unicodedata
import re

In [2]:
def normalize_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'\s+', ' ', text)  
    text = text.strip()
    return text

In [3]:
# Criar o DataFrame e limpar dados
df = pd.read_csv('tweets_coletados.csv').dropna()
df = df[~df['username'].str.contains('@SorvetesFrosty', case=False)]

In [4]:
df.head()

,id,username,name,date,text,url
0,1853110476390936816,@NdeNathan,Nathan,2024-11-03 16:22:21,Vocês preferem sorvete Frosty ou Frut Biss????...,https://twitter.com@NdeNathan/status/185311047...
1,1849103750952026274,@ange1lindo,xeixo sentido,2024-10-23 15:01:03,parece q essa poha tem emulsificante que quand...,https://twitter.com@ange1lindo/status/18491037...
2,1829598844570734968,@naionai_,evaristo,2024-08-30 19:15:31,negócio da pardal é q eu acho aguado os sorvet...,https://twitter.com@naionai_/status/1829598844...
3,1826071964475105398,@lucsisokay,lucas guadagnino,2024-08-21 01:40:57,sorvete napolitano zero da frosty maior invenç...,https://twitter.com@lucsisokay/status/18260719...
4,1825321740962025584,@rmpelomundo,O nômade…,2024-08-18 23:59:50,Coming um pote de sorvete da Frosty quase todo...,https://twitter.com@rmpelomundo/status/1825321...


In [5]:
# Correção dos links dos tweets
if 'url' in df.columns:
    df['url'] = df['url'].str.replace('twitter.com@', 'twitter.com/', regex=False)

In [6]:
# Dicionário de sabores e palavras-chave
flavors = {
    'morango': ['morango'],
    'flocos': ['flocos'],
    'chocolate': ['chocolate'],
    'leitinho trufado': ['leitinho', 'leitinho trufado'],
    'napolitano': ['napolitano'],
    'bombom': ['bombom'],
    'chocolate zero': ['chocolate zero'],
    'napolitano zero': ['napolitano zero'],
    'três chocolates': ['três chocolates', '3 chocolates', 'três chocolates'],
    'mousse de maracujá': ['maracuja', 'mousse de maracuja', 'maracujá'],
    'grego com amarena': ['grego com amarena'],
    'delícia de abacaxi': ['abacaxi'],
    'creme brulée': ['creme brulee', 'crème brulée', 'brule', 'brulee', 'brulée'],
    'supermaltine': ['supermaltine', 'maltine', 'ovomaltine'],
    'chocolate trufado': ['chocolate trufado', 'chocolate gold'],
    'castanha com paçoca': ['castanha com paçoca'],
    'tapioca': ['tapioca'],
    'pavê': ['pavê', 'pave'],
    'nata goiaba': ['nata goiaba'],
    'napolitano tropical': ['napolitano tropical'],
    'brigadeiro': ['brigadeiro'],
    'creme': ['creme'],
    'creme com passas': ['creme com passas'],
    'açaí': ['açai', 'açaí', 'assai', 'acai']
}

In [7]:
# Criar uma lista de palavras-chave ordenadas por tamanho para evitar capturas parciais
keyword_flavor_pairs = []
for flavor, keywords in flavors.items():
    for kw in keywords:
        keyword_flavor_pairs.append((normalize_text(kw), flavor))

# Ordenar pela palavra-chave mais longa primeiro, para evitar capturas incorretas
keyword_flavor_pairs.sort(key=lambda x: len(x[0]), reverse=True)

In [ ]:
# Função para detectar sabores sem capturas duplicadas
def detectar_sabores(texto):
    texto_normalizado = normalize_text(texto)
    sabores_detectados = set()
    
    for kw, sabor in keyword_flavor_pairs:
        if kw in texto_normalizado:
            if not any(s in sabores_detectados for s in flavors.keys() if kw in normalize_text(s)):
                sabores_detectados.add(sabor)
    
    return ', '.join(sabores_detectados) if sabores_detectados else 'nulo'

df['sabores'] = df['text'].apply(detectar_sabores)


In [9]:
# Análise de sentimento
positive_words = [
    'bom', 'boa', 'ótimo', 'excelente', 'maravilhoso', 'gostei', 'adoro', 'amei', 'incrível', 
    'recomendo', 'perfeito', 'fantástico', 'sensacional', 'top', 'show', 'excepcional', 'delicioso', 
    'saboroso', 'cremoso', 'suave', 'refrescante', 'divino', 'espetacular', 'agradável', 'viciante', 
    'irresistível', 'melhor', 'nota 10', 'imperdível', 'inesquecível', 'sofisticado', 'harmonioso', 
    'caprichado', 'apetitoso', 'muito bom', 'superior', 'de qualidade', 'caprichado', 'cheio de sabor', 
    'textura perfeita', 'combinação perfeita', 'chocolate intenso', 'doce na medida certa', 
    'vale cada centavo', 'de outro nível', 'bem feito', 'impecável', 'sensação única', 'satisfatório', 
    'dá vontade de comer mais', 'leve e gostoso', 'surpreendente', 'encantador', 'reconfortante', 
    'frescura na medida', 'melhor do mercado', 'premium', 'artesanal', 'equilibrado', 'crocante', 
    'macio na medida', 'delicadeza no sabor', 'impressionante', 'puro prazer', 'cheiro delicioso', 
    'camadas incríveis', 'bem recheado', 'vale a pena', 'simplesmente maravilhoso', 'cheio de pedacinhos', 
    'textura sedosa', 'rico em sabor', 'combinação incrível', 'explosão de sabor', 'muito gostoso', 
    'não tem erro', 'ponto certo', 'quentinho por dentro', 'de lamber os dedos', 'derrete na boca', 
    'de outro mundo', 'não dá para resistir', 'simplesmente perfeito', 'o melhor sorvete', 
    'quentinho e cremoso', 'chocolate derretendo', 'muito cremoso', 'equilíbrio perfeito', 
    'sabor inesquecível', 'chega a derreter na boca', 'bem gelado e delicioso', 'textura impecável', 
    'vale cada mordida', 'combinação dos deuses', 'delícia pura', 'top de linha', 'de alta qualidade', 
    'o verdadeiro sabor', 'sabor bem presente', 'gostinho de infância', 'sabor marcante', 
    'de primeira', 'fechou com chave de ouro', 'um espetáculo de sabor', 'vontade de repetir', 
    'difícil de parar de comer'
]

negative_words = [
    'ruim', 'péssimo', 'horrível', 'odeio', 'detestei', 'terrível', 'decepcionante', 'lixo', 
    'não gostei', 'horroroso', 'fracasso', 'desastre', 'insatisfeito', 'pessimo', 'nojento', 
    'azedo', 'aguado', 'sem gosto', 'mole demais', 'duro demais', 'grudento', 'pastoso', 
    'enjoativo', 'insuportável', 'intragável', 'repugnante', 'medíocre', 'sem graça', 'sem sabor', 
    'muito doce', 'artificial', 'gosto estranho', 'químico', 'rançoso', 'cheiro ruim', 'estragado', 
    'parece plástico', 'péssima textura', 'desmancha rápido', 'grãos de gelo', 'sem cremosidade', 
    'pouco recheio', 'enganoso', 'fraude', 'dinheiro jogado fora', 'pior escolha', 'indigesto', 
    'faz mal', 'pesado', 'mal feito', 'pior sorvete', 'propaganda enganosa', 'corante demais', 
    'adoçante ruim', 'com gosto de papelão', 'quase sem chocolate', 'caro e ruim', 'não recomendo', 
    'pessimamente feito', 'cheio de cristais de gelo', 'cheio de gordura hidrogenada', 
    'parece manteiga', 'sem qualidade', 'derrete rápido', 'péssima embalagem', 'chega todo derretido', 
    'não vale a pena', 'frustrante', 'desagradável', 'textura bizarra', 'muito pesado', 'muito enjoativo', 
    'sensação gordurosa', 'parece sabão', 'parece isopor', 'nada especial', 'sem diferencial', 
    'totalmente esquecível', 'não dá vontade de comer', 'péssima experiência', 'não vale o preço', 
    'exageradamente doce', 'excesso de essência', 'parece farinha', 'gosto de talco', 'tem gosto de nada', 
    'amargo', 'parece chantilly barato', 'consistência estranha', 'cheio de pedaços duros', 
    'poderia ser melhor', 'gosto de remédio', 'parece feito de plástico', 'simplesmente horrível', 
    'não dá para comer', 'muito artificial', 'sensação estranha na boca', 'qualidade duvidosa', 
    'recheio falso', 'propaganda exagerada', 'decepcionou', 'parece velho', 'parece estar vencido', 
    'não presta', 'parece água congelada'
]

positive_norm = [normalize_text(w) for w in positive_words]
negative_norm = [normalize_text(w) for w in negative_words]

def analisar_sentimento(texto):
    texto_norm = normalize_text(texto)
    pos = sum(1 for w in positive_norm if w in texto_norm)
    neg = sum(1 for w in negative_norm if w in texto_norm)
    return 'positivo' if pos > neg else 'negativo' if neg > pos else 'neutro'

df['sentimento'] = df['text'].apply(analisar_sentimento)

In [10]:
df.head()

,id,username,name,date,text,url,sabores,sentimento
0,1853110476390936816,@NdeNathan,Nathan,2024-11-03 16:22:21,Vocês preferem sorvete Frosty ou Frut Biss????...,https://twitter.com/NdeNathan/status/185311047...,nulo,neutro
1,1849103750952026274,@ange1lindo,xeixo sentido,2024-10-23 15:01:03,parece q essa poha tem emulsificante que quand...,https://twitter.com/ange1lindo/status/18491037...,nulo,positivo
2,1829598844570734968,@naionai_,evaristo,2024-08-30 19:15:31,negócio da pardal é q eu acho aguado os sorvet...,https://twitter.com/naionai_/status/1829598844...,nulo,negativo
3,1826071964475105398,@lucsisokay,lucas guadagnino,2024-08-21 01:40:57,sorvete napolitano zero da frosty maior invenç...,https://twitter.com/lucsisokay/status/18260719...,napolitano zero,neutro
4,1825321740962025584,@rmpelomundo,O nômade…,2024-08-18 23:59:50,Coming um pote de sorvete da Frosty quase todo...,https://twitter.com/rmpelomundo/status/1825321...,nulo,neutro


In [11]:
df.to_csv('tweets_processados.csv', index=False)